<a href="https://colab.research.google.com/github/w-e-s-l-e-y/BairesDev---Machine-Learning-Practitioner/blob/main/reconhecimentoFacial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Desinstala qualquer versão do OpenCV para garantir uma instalação limpa
!pip uninstall opencv-python -y
!pip uninstall opencv-contrib-python -y

# Força a desinstalação do opencv-python (mesmo que não esteja instalado)
!pip uninstall opencv-python -y

# Instala o OpenCV com os módulos contrib
!pip install opencv-contrib-python

Found existing installation: opencv-contrib-python 4.11.0.86
Uninstalling opencv-contrib-python-4.11.0.86:
  Successfully uninstalled opencv-contrib-python-4.11.0.86
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (69.1 MB)


In [5]:
# --- 1. Importação de Dependências ---
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import os

# --- 2. Funções Auxiliares ---

# Função para converter o objeto JavaScript (imagem da webcam) em uma imagem OpenCV (BGR).
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # Decodifica a imagem base64
  image_bytes = b64decode(js_reply.split(',')[1])
  # Converte bytes para um array NumPy
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # Decodifica o array NumPy em uma imagem OpenCV BGR
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# Função para converter uma caixa delimitadora (bbox) do OpenCV em uma string de bytes base64.
#   Esta função agora também converte a imagem final.
def image_and_bbox_to_bytes(img, bbox_array):
  """
  Converte a imagem e a sobreposição em uma única string de bytes base64.

  Params:
      img: Imagem original (OpenCV BGR).
      bbox_array: Array NumPy com a sobreposição (caixa delimitadora e nome).
  Returns:
      bytes: String de bytes base64 da imagem final.
  """
  # Combina a imagem original com a sobreposição
  bbox_array = bbox_array.astype(np.uint8)  # Garante o tipo correto
  img_with_bbox = cv2.addWeighted(img, 1, cv2.cvtColor(bbox_array, cv2.COLOR_RGBA2BGR), 0.5, 0)

  # Converte para PNG
  _, buffer = cv2.imencode('.png', img_with_bbox)
  iobuf = io.BytesIO(buffer)
  img_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return img_bytes

# --- 3. Classificador Haar Cascade e Reconhecimento de Faces ---

# Inicializa o modelo Haar Cascade para detecção de faces.
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

# Inicializa o reconhecedor de faces LBPH
recognizer = cv2.face.LBPHFaceRecognizer_create()

# Diretório para armazenar dados de faces
face_data_dir = 'face_data'
if not os.path.exists(face_data_dir):
    os.makedirs(face_data_dir)

# Função para carregar dados de faces conhecidas e treinar o reconhecedor
def load_face_data():
    names = {}
    training_data, labels = [], []

    for name in os.listdir(face_data_dir):
        name_dir = os.path.join(face_data_dir, name)
        if os.path.isdir(name_dir):
            names[len(names)] = name
            for filename in os.listdir(name_dir):
                if filename.endswith(".jpg"):
                    img_path = os.path.join(name_dir, filename)
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                    training_data.append(img)
                    labels.append(list(names.keys())[list(names.values()).index(name)])

    if not training_data:
        print("Nenhum dado de face encontrado. Treinando com uma imagem inicial...")
        # Cria uma imagem dummy (preta) para treinamento inicial
        img = np.zeros((100, 100), dtype=np.uint8)
        training_data.append(img)
        labels.append(0)
        names[0] = "admin"
        # Cria um diretório e salva a imagem dummy
        dummy_dir = os.path.join(face_data_dir, "admin")
        os.makedirs(dummy_dir, exist_ok=True)
        cv2.imwrite(os.path.join(dummy_dir, "admin_0.jpg"), img)

    if training_data:
        recognizer.train(training_data, np.array(labels))
        print("Treinamento concluído!")
    return names

# Carrega dados de faces conhecidas e treina o reconhecedor inicialmente
names = load_face_data()

# --- 4. Imagens da Webcam --- (Removida a função take_photo)

# --- 5. Vídeos da Webcam ---

# --- Código JavaScript para criar o stream de vídeo ao vivo ---
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    var finalImageElement; // Elemento para a imagem final

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
      if (stream) {
        stream.getVideoTracks().forEach(track => track.stop());
      }
      if (div) { // Verifica se div existe antes de remover
        div.remove();
      }
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
       // finalImageElement não é removido!
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "Status:";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          'When finished, click here or on the video to stop this demo';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      // Cria o elemento <img> para a imagem final (fora do div)
      finalImageElement = document.createElement('img');
      document.body.appendChild(finalImageElement);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        if (imgData != "") { // Exibe a imagem final se houver dados
            finalImageElement.src = imgData;
        }
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

# --- Função para processar cada quadro do vídeo ---
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

def stop_webcam():
    eval_js('removeDom()')

# --- 6. Código Principal (Vídeo) ---

# Inicia o stream de vídeo da webcam *uma vez* no início
video_stream()
label_html = 'Aguardando detecção...'  # Mensagem inicial
bbox = ''

# Função para capturar uma face da webcam (agora sem iniciar a webcam)
def capture_face_from_webcam():
    # Usa a webcam já ativa
    label_html = 'Olhe para a câmera...'
    bbox = ''

    # Captura alguns quadros para dar tempo do usuário se posicionar
    for _ in range(30):  # Captura ~30 quadros (ajuste conforme necessário)
      js_reply = video_frame(label_html, bbox) # Captura frame
      if not js_reply:
          print("Erro ao capturar quadro.")
          return None
      time.sleep(0.1)  # Pequena pausa entre as capturas

    #Agora vamos pegar o ultimo frame:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
      print("Erro ao capturar o ultimo frame.")
      return None

    # Converte a resposta JS em uma imagem OpenCV
    img = js_to_image(js_reply["img"])

    # Converte a imagem para escala de cinza
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Detecta faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) > 0:
        # Captura a primeira face detectada
        (x, y, w, h) = faces[0]
        face_roi = gray[y:y + h, x:x + w]
        print("Face capturada!")
        return face_roi, img #Retorna a face e a imagem original
    else:
        print("Nenhuma face detectada.")
        return None, None

# --- Loop principal ---
new_face_detected = False
while True:
    if new_face_detected:
      # Captura a face (sem iniciar a webcam)
      face_roi, original_image = capture_face_from_webcam()

      if face_roi is not None:
          # Pergunta o nome e salva a imagem
          name = input("Novo rosto detectado! Insira o nome para treinamento: ")
          new_name_dir = os.path.join(face_data_dir, name)
          if not os.path.exists(new_name_dir):
              os.makedirs(new_name_dir)
          img_path = os.path.join(new_name_dir, f"{name}_{int(time.time())}.jpg")
          cv2.imwrite(img_path, face_roi)

          # Treina o modelo
          print("Treinando modelo com nova face...")
          names = load_face_data()
          label = list(names.keys())[list(names.values()).index(name)]
          recognizer.update([face_roi], np.array([label]))
          print("Treinamento atualizado!")
          #stop_webcam() # Para a webcam após o treinamento. NÃO MAIS NECESSÁRIO AQUI

      new_face_detected = False
      label_html = 'Aguardando detecção...'

    else: #Modo de detecção
      js_reply = video_frame(label_html, bbox) #Mantem a webcam ligada, mas só captura frames para detecção.
      if not js_reply:
        break
      # Converte a resposta JS em uma imagem OpenCV
      img = js_to_image(js_reply["img"])

      # Cria uma sobreposição transparente para a caixa delimitadora
      bbox_array = np.zeros([480,640,4], dtype=np.uint8)

      # Converte a imagem para escala de cinza
      gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

      # Obtém as coordenadas da região da face
      faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

      #Se detectou faces, verifica se são conhecidas
      for (x, y, w, h) in faces:
          face_roi = gray[y:y+h, x:x+w]
          label, confidence = recognizer.predict(face_roi)
          confidence_threshold = 75

          if confidence < confidence_threshold:
              name = names[label]
              cv2.rectangle(bbox_array, (x, y), (x+w, y+h), (255, 0, 0), 2)
              cv2.putText(bbox_array, name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
          else: #Rosto desconhecido
              new_face_detected = True #Seta a flag para entrar no modo de captura
              break #Sai do loop for, pois já detectou um rosto desconhecido

      if not new_face_detected:  # Só exibe se não for um novo rosto
        # Converte a imagem e a sobreposição para bytes
        img_bytes = image_and_bbox_to_bytes(img, bbox_array)
        bbox = img_bytes  # Atualiza bbox com a imagem completa
      else:
        bbox = ""

Treinamento concluído!


<IPython.core.display.Javascript object>